In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import copy as copy
from homebrewedFunctions.functions import *

In [2]:


data_cats = ["NoTaxRatios","WithTaxRatios"]
dfs_dct = {}
for data_cat in data_cats:
    filename = f"panelData{data_cat}.xlsx"
    dfs_dct[data_cat] = pd.read_excel(filename,sheet_name=None)
    for key, df in dfs_dct[data_cat].items():
        dfs_dct[data_cat][key].set_index(["State","DATE"], inplace = True)
        del dfs_dct[data_cat][key]["Unnamed: 0"]
dfs_dct

{'NoTaxRatios': {'Levels':                   Population  Home Price Index  EFNA  Overall-scores(allgovs)  \
  State DATE                                                                      
  AL    1900-01-01     1830.00               NaN   NaN                      NaN   
        1901-01-01     1907.00               NaN   NaN                      NaN   
        1902-01-01     1935.00               NaN   NaN                      NaN   
        1903-01-01     1957.00               NaN   NaN                      NaN   
        1904-01-01     1978.00               NaN   NaN                      NaN   
  ...                      ...               ...   ...                      ...   
  WY    2019-01-01      580.12            324.49  8.16                 8.156352   
        2020-01-01      577.66            340.92  8.01                 8.009364   
        2021-01-01      579.55            380.47  7.99                 7.988226   
        2022-01-01      581.63            438.83   NaN        

In [3]:
plot_dfs = {"Level":dfs_dct["NoTaxRatios"]["Levels"].copy(),
           "Percent Total":dfs_dct["NoTaxRatios"]["Levels"].copy()}
keys = list(plot_dfs["Level"].keys())
GDP_components = keys[13:18]
tax_components = keys[20:-1]
ind_tax_components = [t for t in tax_components if "(" in t]
prime_GDP_components = copy.copy(GDP_components)
prime_GDP_components.pop(1)
states = plot_dfs["Level"].index.get_level_values("State").unique()
years = plot_dfs["Level"].loc["AL"].index.unique()
for lr in ["Level", "Percent Total"]:
    plot_dfs[lr][tax_components + ["Total Taxes"]]= plot_dfs[lr][tax_components + ["Total Taxes"]].mul(10**3)
    plot_dfs[lr][["GDP"] + GDP_components] = plot_dfs[lr][["GDP"] + GDP_components].mul(10**6)
plot_dfs["Percent Total"][tax_components] = plot_dfs["Level"][tax_components].apply(lambda x: x.div(plot_dfs["Level"]["Total Taxes"])).mul(100)
plot_dfs["Percent Total"][GDP_components] = plot_dfs["Level"][GDP_components].apply(lambda x: x.div(plot_dfs["Level"]["GDP"])).mul(100)
plot_dfs["Percent GDP"] = plot_dfs["Level"].copy()
plot_dfs["Percent GDP"][tax_components + ["Total Taxes"] + GDP_components] = plot_dfs["Level"][
    tax_components + ["Total Taxes"] + GDP_components].apply(lambda x: x.div(plot_dfs["Level"]["GDP"])).mul(100)
panel_dfs = {}
for key, df in plot_dfs.items():
    panel = df.reset_index().melt(id_vars=["State","DATE"], 
        var_name="Name", 
        value_name="Value")
    panel = panel.set_index(["State", "DATE"]).round(2)
    panel_dfs[key] = panel.copy()
panel_dfs

{'Level':                             Name    Value
 State DATE                               
 AL    1900-01-01      Population  1830.00
       1901-01-01      Population  1907.00
       1902-01-01      Population  1935.00
       1903-01-01      Population  1957.00
       1904-01-01      Population  1978.00
 ...                          ...      ...
 WY    2019-01-01  GDP Per Capita    69.68
       2020-01-01  GDP Per Capita    62.74
       2021-01-01  GDP Per Capita      NaN
       2022-01-01  GDP Per Capita      NaN
       2023-01-01  GDP Per Capita      NaN
 
 [328600 rows x 2 columns],
 'Percent Total':                             Name    Value
 State DATE                               
 AL    1900-01-01      Population  1830.00
       1901-01-01      Population  1907.00
       1902-01-01      Population  1935.00
       1903-01-01      Population  1957.00
       1904-01-01      Population  1978.00
 ...                          ...      ...
 WY    2019-01-01  GDP Per Capita    69.6

In [4]:
df = panel_dfs["Level"]
df["HoverText"] = df["Name"] + ":<br>" + df["Value"].astype(str)
# df = df[df.index.get_level_values("DATE") == "2020"]
names = df["Name"].unique()
df_pivot = df.reset_index().pivot(index=['State', 'DATE', "HoverText"], columns='Name', values='Value').reset_index()
df_pivot


Name,State,DATE,HoverText,3-allg,3A-allg,3B-allg,3C-allg,4-allg,5-allg,6-allg,...,State and Local Spending,Taxes NEC (T99),Tobacco Tax (T16),Tot Sales & Gr Rec Tax,Total Gen Sales Tax (T09),Total Income Taxes,Total License Taxes,Total Other Taxes,Total Select Sales Tax,Total Taxes
0,AK,1900-01-01,3-allg:<br>nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,1900-01-01,3A-allg:<br>nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,1900-01-01,3B-allg:<br>nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AK,1900-01-01,3C-allg:<br>nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AK,1900-01-01,4-allg:<br>nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328595,WY,2023-01-01,Total Income Taxes:<br>0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
328596,WY,2023-01-01,Total License Taxes:<br>219407000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,219407000.0,NaN,NaN,NaN
328597,WY,2023-01-01,Total Other Taxes:<br>1155472000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.155472e+09,NaN,NaN
328598,WY,2023-01-01,Total Select Sales Tax:<br>198569000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198569000.0,NaN


In [80]:
import plotly.express as px
import pandas as pd

def create_map(df, name):
    plot_df = df[['State', 'DATE', name]].dropna()
    dates = sorted([str(d)[:4] for d in plot_df["DATE"].unique()])
    dates_dict[name] = dates
    plot_df["DATE"] = plot_df["DATE"].astype(str).str[:4]
    plot_df = plot_df.reset_index().pivot(index=['State'], columns='DATE', values=name).reset_index()

    init_var = dates[-1]

    # Calculate the overall min and max values across all years
    min_val = plot_df.drop(columns='State').min().min()
    max_val = plot_df.drop(columns='State').max().max()

    # Create the initial plot
    fig = px.choropleth(plot_df,
                        locations='State',
                        color=init_var,  # Initial variable
                        color_continuous_scale='spectral_r',
                        locationmode='USA-states',
                        scope='usa')
    # Update the coloraxis to match the overall min and max
    fig.update_traces(hovertemplate="%{location}: %{z}<extra></extra>", 
                    selector=dict(type='choropleth'))
    fig.update_layout(
        coloraxis_colorbar=dict(title=''),
        coloraxis=dict(cmin=min_val, cmax=max_val, colorscale='spectral_r')
        )
    # Update layout with slider
    sliders = [dict(
        active=len(dates)-1,
        currentvalue={"prefix": "Year: "},
        pad={"t": 50},
        steps=[
            dict(
                label=date,
                method="update",
                args=[
                    {"z": [plot_df[date]]},
                    {"hovertemplate": "%{location}: " + plot_df[date].astype(str) + "<extra></extra>"},
                    {"title": f"{name}: {date}"}
                ]
            ) for date in dates
        ]
    )]
    fig.update_layout(
        sliders=sliders,
        title_text=name, title_x=0.5,
        coloraxis_colorbar=dict(title=''),
        coloraxis=dict(cmin=min_val, cmax=max_val, colorscale='spectral_r')
        )

    return fig

def combine_map_figs(figs):
    # Combine all figures into one figure with a menu to select between them
    combined_fig = make_subplots(rows=1, cols=1)
    names = list(figs.keys())
    for name, fig in figs.items():
        for trace in fig.data:
            combined_fig.add_trace(trace)
    # Create dropdown menu to switch between figures
    dropdown_buttons = [
        dict(
            label=name,
            method="update",
            args=[{"visible": [True if name == selected_name else False for selected_name in names]},
                {**figs[name].layout.to_plotly_json()},
                {"title": name}]
        ) for name in names
    ]
    combined_fig.layout = {**figs[list(figs.keys())[0]].layout.to_plotly_json()}
    combined_fig.update_layout(
        updatemenus=[dict(buttons=dropdown_buttons, direction="down", showactive=True)],
        title_text=names[0], title_x=0.5
    )

    # Set visibility for initial state
    combined_fig.for_each_trace(lambda trace: trace.update(visible=False))
    combined_fig.data[0].update(visible=True)
    return combined_fig



In [81]:
figs = {}
dates_dict = {}

for name in names:
    fig = create_map(df_pivot, name)
    figs[name] = fig
combined_fig = combine_map_figs(figs)
# combined_fig.show()
combined_fig.write_html("outputs/AllMapsByVariableAndYear.html")

## alternate: present all variables within a given year.

In [6]:
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
# plot_df = df_pivot[df_pivot["DATE"] == "2020"]


# # Pivot the dataframe
# init_var = "Population"

# # Create the initial plot
# fig = px.choropleth(plot_df,
#                     locations='State',
#                     color=init_var,  # Initial variable
#                     color_continuous_scale='spectral_r',
#                     hover_name="HoverText",
#                     locationmode='USA-states',
#                     scope='usa')
# fig.update_traces(hovertemplate="%{hovertext}<extra></extra>")

# # Update layout for dropdown
# fig.update_layout(
#     updatemenus=[
#         dict(
#             buttons=list([
#                 dict(label=name,
#                      method="update",
#                      args=[{"z": [plot_df[name]]},
#                            {"title":name},
                           
#                         #    {"hovertemplate": '<b>%{name}:</b><br>% {z}<extra></extra>'}
#                            ]) for name in names
#                 # Add more variables here if needed
#             ]),
#             direction="down",
#             pad={"r": 10, "t": 10},
#             showactive=True,
#             x=0.1,
#             xanchor="left",
#             y=1.3,
#             yanchor="top"
#         ),
#     ],
#     title_text=init_var, title_x=0.5,
#                   coloraxis_colorbar=dict(title=''))

# fig.update_layout()

# fig.show()
